# Loading the EAGLE simulation data

Write out the function for loading the EAGLE simulation data.  It checks whether reduced resolution versions have been created already and loads those if it has.

In [11]:
import numpy as np
import eagle_constants_and_units as c
import cosmo_utils as csu
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import mpl_toolkits.axes_grid1 as axgrid
from astropy import constants as const
from astropy import units as u

import os

import get_halpha_SB

In [32]:
def initstuff():
    if machine=='chinook':
        homedir='/Users/lokhorst/Eagle/'
    elif machine=='coho':
        homedir='/Users/deblokhorst/eagle/SlicesFromNastasha/'

    files_SF_28 = [homedir+'emission_halpha_L0100N1504_28_test2_SmAb_C2Sm_32000pix_5.000000slice_zcen12.5__fromSFR.npz',
                    homedir+'emission_halpha_L0100N1504_28_test2_SmAb_C2Sm_32000pix_5.000000slice_zcen17.5__fromSFR.npz',
                    homedir+'emission_halpha_L0100N1504_28_test2_SmAb_C2Sm_32000pix_5.000000slice_zcen2.5__fromSFR.npz',
                    homedir+'emission_halpha_L0100N1504_28_test2_SmAb_C2Sm_32000pix_5.000000slice_zcen7.5__fromSFR.npz']

    files_noSF_28 = [homedir+'emission_halpha_L0100N1504_28_test2_SmAb_C2Sm_32000pix_5.000000slice_zcen12.5_noSFR.npz',
                    homedir+'emission_halpha_L0100N1504_28_test2_SmAb_C2Sm_32000pix_5.000000slice_zcen17.5_noSFR.npz',
                    homedir+'emission_halpha_L0100N1504_28_test2_SmAb_C2Sm_32000pix_5.000000slice_zcen2.5_noSFR.npz',
                    homedir+'emission_halpha_L0100N1504_28_test2_SmAb_C2Sm_32000pix_5.000000slice_zcen7.5_noSFR.npz']

    sl = [slice(None,None,None), slice(None,None,None)]
    
    return homedir,files_SF_28,files_noSF_28,sl

def changeres(distance,resolution,data):
    pixscale =  {'50Mpc': 0.237/1000.*(1.+0.0115), '100Mpc': 0.477/1000.*(1.+0.0235),'200Mpc': 0.928/1000.*(1.+0.047) , '500Mpc': 2.178/1000.*(1.+0.12)} ### Mpc / arcsec (comoving)
    if distance in pixscale:
        simpixsize = 100./32000. ### Mpc / pixel is resolution of raw data 
        factor = round(pixscale[distance]*resolution/simpixsize)
        size = 32000.
        print("Will reduce resolution by a factor of %s."%factor)
        # LATER determine the current resolution of the data. FOR NOW assume current resolution is 100 Mpc/ 32000 pixels ~ 3 kpc/pixel

        # If the factors are not integer multiples of 32000., I'll trim the data first and then imreduce it
        if 32000.%((factor)) != 0.:
            times_factor_fits_in = int(32000./factor)
            newsize = times_factor_fits_in * factor
            print("Before reducing resolution, the original data was trimmed to size %s."%newsize)
            datanew = data[0:int(newsize),0:int(newsize)]
        else:
            datanew = data
            newsize = size
    else:
        print('distance needs to be one of: 50Mpc, 100Mpc, 200Mpc, or 500Mpc.  Others not supported atm.')
        return None
        
    return get_halpha_SB.imreduce(datanew, round(factor), log=True, method = 'average'), newsize, factor


def loadraw(index=0):
    print('data1 ('+files_noSF_28[index]+')...')
    data1 = (np.load(files_noSF_28[index])['arr_0'])[sl]
    #data1 = get_halpha_SB.imreduce(data1, round(factor), log=True, method = 'average')
    print('data11 ('+files_SF_28[index]+')...')
    data11 = (np.load(files_SF_28[index])['arr_0'])[sl]
    #data11 = get_halpha_SB.imreduce(data11, round(factor), log=True, method = 'average')
    print('5 Mpc slice...')
    data = np.log10(10**data1+10**data11)
    print('delete data1, data11...')
    del data1
    del data11
    return data

def load5Mpcslice():
    print('Loading a 5Mpc slice of data...')        
    'check if the SF plus noSF data file already exists...'
    total_fname = 'emission_halpha_L0100N1504_28_test2_SmAb_C2Sm_32000pix_5.000000slice_zcen12.5_total.npz'
    if os.path.isfile(total_fname):
        print("data exists, loading %s now..."%total_fname)
        data = (np.load(total_fname)['arr_0'])[sl]
    else:
        print("data not saved, loading from original files now...")
        data = loadraw()
        np.savez(total_fname,data)
    return data

def loaddata(machine='coho',resolution=None, distance=None, slicewidth=5):
    """
    Returns a tuple containing the data in the first index
    It contains the size of the data before resolution was reduced in the second index (just its own size if raw data)
    It contains the factor by which the data was reduced in the third index (just '1' if raw data)
    """
    homedir,files_SF_28,files_noSF_28,sl = initstuff()
    # Simulation snapnum 28 (z = 0), xy box size: 100Mpc, z slice width: 5Mpc,
    if resolution is None and distance is None and int(slicewidth) == 5:
        data = load5Mpcslice()
        data = np.array([data,data.shape[0],1]) # make into a tuple
        
    elif resolution is None and int(slicewidth) != 5:
        numslices = round(slicewidth / 5.)
        slicewidth = int(numslices * 5.)
        total_fname = 'emission_halpha_L0100N1504_28_test2_SmAb_C2Sm_32000pix_%sslice_zcen12.5_total.npz'%slicewidth
        if os.path.isfile(total_fname):
            print("Data exists, loading %s now..."%total_fname)
            data = (np.load(total_fname)['arr_0'])[sl]
        else:
            print('Creating data of slicewidth %s Mpc...'%slicewidth)
            data = 0
            for index in range(numslices):
                rawdata = load5Mpcslice(index=index)
                data = data + rawdata
            np.savez(total_fname,data)    
        data = np.array([data,data.shape[0],1]) # make into a tuple

    elif resolution is not None and int(slicewidth) == 5:
    # Note: Once the data has new resolution, it is saved as a tuple, with extra info
    # The format of the tuple is: data, trimmed_data_size_before_reducing, factor_by_which_reduced
        if distance is None:
            print('Need to input both a distance and a resolution. Exiting.')
            return None
        fname = 'data_%s_%sarcsec.npz'%(str(distance),int(resolution))
        if os.path.isfile(fname):
            print('%s exists, loading now...')
            data_tuple = (np.load(fname)['arr_0'])
        else:
            print('%s does not exists, making it now...')
            rawdata = load5Mpcslice()
            data_tuple = changeres(distance,resolution,rawdata)
        data = data_tuple # save the tuple to data
        
    return data

In [34]:
### EXAMPLES ###
#machine='coho'

#distance = '50Mpc'; resolution = 100
#data_tuple = loaddata(machine=machine,resolution=resolution,distance=distance)
#data = data_tuple[0]

#data_tuple = loaddata(machine=machine)
#data = data_tuple[0]
######

Loading a 5Mpc slice of data...
data exists, loading emission_halpha_L0100N1504_28_test2_SmAb_C2Sm_32000pix_5.000000slice_zcen12.5_total.npz now...
